In [1]:
import sympy as sym
import numpy as np
import pandas as pd
from scipy.linalg import eig
from scipy.optimize import fsolve
from sympy.physics.mechanics import msubs

In [2]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))

import ModelFramework as mf
import FwtModels.RectWing as rw

## Create Method to Calculate the Eigen Response of a Given model

Model defined by number of b and t shapes and model ID

Using settings for JEC Wing

In [3]:
def Model_Eigen(model_num):
    p = rw.base_params(2)
    sm = mf.SymbolicModel.from_file(f'2DoF-M{model_num}.py')
    
    dataset_name = '2DoF'
    # Set JEC Specific parameters
    p.ratio_fwt.value = 1
    p.rho.value = 1.225
    p.s_t.value = 1
    p.c.value = 0.15
    p.EI.value = 61
    p.rho_t.value = 20/3
    p.e_0.value = 0
    p.e_1.value = 0
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,40,81))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))

    variables = [k for k,v in vars_ls]
    
    calc_fixed = True if np.isin(2,np.array([1,2,3])) else False
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed)   
    flutdf.to_pickle(f'Eigen_DoF2-M{model_num}_{dataset_name}.pkl')

In [7]:
Model_Eigen(0)
Model_Eigen(1)
Model_Eigen(2)
Model_Eigen(3)
Model_Eigen(4)

/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [5]:
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

for k in range(5):
    pool.apply_async(Model_Eigen,args=(k))
pool.close()
pool.join()